# Training A Neural Network to detect multimodal discrimination

## Setup + Install Packages

We use Vision transformers [ViT] to

In [1]:
! pip install -U transformers torch

## Importing dataset

In [2]:
import os
import torch
from transformers import BertModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

In [3]:
# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
# directory where all files will be located
WORKING_DIRECTORY="/content/"

In [5]:
file = WORKING_DIRECTORY + "labeled_data.csv"

In [6]:
df = pd.read_csv(file)

In [7]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


## Make a Dataset and Split

### Take English Values

### Define DataLoader

In [8]:
class CustomDataset(Dataset):
    def __init__(self, df):
      self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        text = row["tweet"]

        # give scaled outputs
        count = int(row["count"])
        outputs = np.array([0, 0], dtype=np.float32)

        if int(row["class"]) <= 1:
            outputs[0] = 1
        else:
            outputs[1] = 1

        return text, outputs

ds = CustomDataset(df)

In [9]:
inps = ds[0]
inps

("!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
 array([0., 1.], dtype=float32))

In [10]:
BATCH_SIZE = 16

dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True)

In [11]:
len(dl)

1549

## Make Model

In [12]:
tokenizer=  AutoTokenizer.from_pretrained("bert-base-uncased")

In [13]:
from transformers import AutoModelForSequenceClassification
class HateDetector(torch.nn.Module):
  def __init__(self):
    super(HateDetector, self).__init__()
    self.text_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device).train()
    self.output_func = torch.nn.Softmax(dim=-1)


  def forward(self, tokens: torch.Tensor, token_attention_mask: torch.Tensor):
    return self.output_func(self.text_model(tokens, attention_mask=token_attention_mask).logits)


### Load weights

In [14]:
model = HateDetector()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.train()

HateDetector(
  (text_model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=7

### Check that model works

In [16]:
inps, outs = ds[0]

In [17]:
tokenized_inps = tokenizer(inps, return_tensors="pt")
tokenized_inps

{'input_ids': tensor([[  101,   999,   999,   999, 19387,  1030,  9815, 19454, 21818,  2135,
          1024,  2004,  1037,  2450,  2017,  5807,  1005,  1056, 17612,  2055,
          9344,  2039,  2115,  2160,  1012,  1004, 23713,  1025,  2004,  1037,
          2158,  2017,  2323,  2467,  2202,  1996, 11669,  2041,  1012,  1012,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [18]:
out = model(tokenized_inps.input_ids.to(device), tokenized_inps.attention_mask.to(device))
out

tensor([[0.6185, 0.3815]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [19]:
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-3)

## Train

In [23]:
def loss_function(y_pred: torch.Tensor, y_true: torch.Tensor, ce = torch.nn.BCELoss(reduction="none")):
  temp = ce(y_pred, y_true).sum(dim=-1)
  weights = torch.where(y_pred[:, 1] == 1, 1, .1)
  return (temp * weights).mean()


In [24]:
def train(epochs: int):
  model.train()
  for epoch in range(epochs):
    print("Epoch", epoch)
    iterable = iter(dl)

    for batch in (prog := tqdm(range(len(dl)))):
      optimizer.zero_grad()

      # get data
      batch_data = next(iterable)
      inps, labels = batch_data
      tokenized_inps = tokenizer(inps, padding=True, return_tensors='pt', max_length=512, truncation=True)

      # call model and calculate loss
      out = model(tokenized_inps.input_ids.to(device), tokenized_inps.attention_mask.to(device))
      loss = loss_function(out, labels.to(device))

      # backpropagate loss
      loss.backward()
      optimizer.step()

      # log
      prog.set_postfix_str(f"loss: {loss.detach().cpu().item():.5f}")


In [25]:
train(1)

Epoch 0


  0%|          | 0/1549 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

### Save model

In [38]:
from huggingface_hub import notebook_login
notebook_login()

In [39]:
model.text_model.push_to_hub("bert-discrimination-classifier")

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/chreh/bert-discrimination-classifier/commit/c3429280d815f866a4fffe15b5ad04bf6583f50d', commit_message='Upload BertForSequenceClassification', commit_description='', oid='c3429280d815f866a4fffe15b5ad04bf6583f50d', pr_url=None, pr_revision=None, pr_num=None)

# View Results

In [51]:
# 0 - Hate / Offensive, 1 - Neither
MEANINGS = ["Offensive Language", "Neither"]

def get_scores(prompt) -> np.array:
  tokenized_inputs = tokenizer(prompt, return_tensors="pt")
  with torch.no_grad():
    scores = model(tokenized_inputs.input_ids.to(device),
                   tokenized_inputs.attention_mask.to(device)
    )
  return scores[0].detach().cpu().numpy()

def get_raw_score_meanings(scores: np.ndarray) -> str:
  return ", ".join(map(lambda m, score: m + f": {int(score*100)}%", MEANINGS, scores)).strip(", ")

def get_processed_score_meanings(scores: np.ndarray) -> str:
  if scores[0] < 0.5:
    return "This is not a hateful post."
  else:
    return "This is a hateful post."

def pipeline(prompt):
  scores = get_scores(prompt)
  print("Raw scores:")
  print(get_raw_score_meanings(scores))
  print()
  print(get_processed_score_meanings(scores))


In [52]:
prompt = (
"""
I love my class! I learn so much stuff that will be super useful!
"""
)

In [53]:
pipeline(prompt)

Raw scores:
Offensive Language: 19%, Neither: 80%

This is not a hateful post.


In [54]:
prompt= "FUCK INDIAN PEOPLE"

In [55]:
pipeline(prompt)

Raw scores:
Offensive Language: 99%, Neither: 0%

This is a hateful post.
